In [24]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver

op = webdriver.ChromeOptions()
op.add_argument('headless')

driver = webdriver.Chrome(executable_path=r"/Users/ninja/Downloads/chromedriver", options=op)
driver.get("https://www.food.com/recipe")
time.sleep(2)  
driver.find_element_by_css_selector('.gk-aa-load-more').click()
scroll_pause_time = 2 
screen_height = driver.execute_script("return window.screen.height;")   
i = 1

while True:
    
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    
    if (screen_height) * i > scroll_height:
        break

soup = BeautifulSoup(driver.page_source, "html.parser")
with open("output1.html", "w") as file:
    file.write(str(soup))

/var/folders/lw/4hhwbl4d4q1dfpb0z8l91sr40000gn/T/ipykernel_55263/552557789.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"/Users/ninja/Downloads/chromedriver", options=op)
/var/folders/lw/4hhwbl4d4q1dfpb0z8l91sr40000gn/T/ipykernel_55263/552557789.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('.gk-aa-load-more').click()


In [1]:
# from dataprep.connector import connect

# anime_connector = connect("./food")
# ranked_animes = await anime_connector.query('food', q='topic:python+topic:distributed+topic:machine-learning')
print('test')
# print(ranked_animes)

    

NameError: name 'connect' is not defined

In [ ]:

from bs4 import BeautifulSoup

In [1]:
from dataprep.connector import connect
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import asyncio
import math

# "authorization": "Bearer",
# "pagination": {
#             "type": "page",
#             "pageKey": "page",
#             "limitKey": "pageSize",
#             "maxCount": 5000000
#         } 
# "pagination": {
#             "type": "offset",
#             "offsetKey": "f",
#             "limitKey": "h",
#             "maxCount": 100000
#         }   
# Provide your API key here for TAs to reproduce your results
# API_key = "8D73AS9T8U2JKXRP"

connect_api = connect("./config", _concurrency=100)
metadata = await connect_api.query('allrecipes', page=1)
result_set = metadata.iloc[0]['totalResults']
perPage = 24
total_requests = math.ceil(result_set/perPage)
print(total_requests)
async def fetch_records(queries_to_fetch):
    results = asyncio.gather(*queries_to_fetch)
    df = pd.concat(await results)
    return df

# Connecting to Web API using dataprep
queries = []
for i in range(1, total_requests):
    query = connect_api.query('allrecipes', page=i)
    queries.append(query)

df = await fetch_records(queries)

# loop = asyncio.get_event_loop()
# df = loop.create_task(fetch_records(queries))
# query_list = [connect_api.query('allrecipes', page=i) for i in range(int(10))]
# results = asyncio.gather(*query_list)
# inflation_df = pd.concat(await results)

# query_list = await connect_api.query('allrecipes', page=1)

# results = asyncio.gather(*query_list)
# inflation_df = pd.concat(await results)

# initial = True
# page = 1
# while initial or json_data['hasNext']:

NumExpr defaulting to 8 threads.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ninja/Documents/Spring22/CMPT733/whats-cooking/config'

In [20]:
from bs4 import BeautifulSoup

file = open('sample.html')
html = file.read()

soup = BeautifulSoup(html, 'html.parser')
left_pane = soup.find('div', class_='recipe-layout__content-left')

prep_time = left_pane.find('div', class_= 'recipe-facts__time').text
serves = left_pane.find('div', class_='recipe-facts__servings').text
ingredients = left_pane.find_all('li', class_='recipe-ingredients__item')
ingredients = [ingredient.text for ingredient in ingredients]
recipe = soup.find_all('li', class_='recipe-directions__step')
recipe = [step.text for step in recipe]
nutrients = left_pane.find_all('p', class_='recipe-nutrition__item')
nutrients = [nutrient.text for nutrient in nutrients]
print(prep_time)
print(serves)
print(ingredients)
print(recipe)
print(nutrients)

READY IN: 35mins
SERVES: 4
[' 2    lbs    boneless chicken breasts, cut into bite-size pieces ', ' 1 -2    tablespoon    olive oil', ' 1   garlic clove, crushed ', ' 1⁄4   teaspoon    ginger', ' 3⁄4   teaspoon    crushed red pepper flakes', ' 1⁄4   cup    apple juice', ' 1⁄3   cup    light brown sugar', ' 2    tablespoons    ketchup', ' 1    tablespoon    cider vinegar', ' 1⁄2   cup    water', ' 1⁄3   cup    soy sauce']
["Editor's Note:  Named Bourbon Chicken because it was supposedly created by a Chinese cook who worked in a restaurant on Bourbon Street.", 'Heat oil in a large skillet.', 'Add chicken pieces and cook until lightly browned.', 'Remove chicken.', 'Add remaining ingredients, heating over medium Heat until well mixed and dissolved.', 'Add chicken and bring to a hard boil.', 'Reduce heat and simmer for 20 minutes.', 'Serve over hot rice and ENJOY.']
['\n          Calories: 521.4\n        ', 'Calories from Fat 220 g 42 %', 'Total Fat 24.5 g 37 %', 'Saturated Fat 6.5 g 32 %', 

In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 

op = webdriver.ChromeOptions()
op.add_argument('headless')
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s, options=op)
# driver = webdriver.Chrome('./chromedriver', options=op)





====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/ninja/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache
Driver [/Users/ninja/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [119]:


def scrape_recipe(URL, recipe_id):
    URL = 'https://www.food.com/recipe/kittencals-easy-and-delicious-ranch-parmesan-chicken-200296'
    driver.get("{}?ftab=reviews".format(URL))
    button = driver.find_elements(by=By.CSS_SELECTOR, value=".conversation__show-more")
    while(len(button) > 0):
        button[0].click()
        time.sleep(1)
        button = driver.find_elements(by=By.CSS_SELECTOR, value=".conversation__show-more")
    page_source = driver.page_source
    return get_data(page_source, recipe_id)



In [120]:
def get_data(page_source, recipe_id):
    reviewer_ids = []
    recipe_ids = []
    review_texts = []

    soup = BeautifulSoup(page_source, 'html.parser')

    left_pane = soup.find('div', class_='recipe-layout__content-left')

    prep_time = left_pane.find('div', class_= 'recipe-facts__time').text
    serves = left_pane.find('div', class_='recipe-facts__servings').text
    ingredients = left_pane.find_all('li', class_='recipe-ingredients__item')
    ingredients = [ingredient.text for ingredient in ingredients]
    recipe = soup.find_all('li', class_='recipe-directions__step')
    recipe = [step.text for step in recipe]
    nutrients = left_pane.find_all('p', class_='recipe-nutrition__item')
    nutrients = [nutrient.text for nutrient in nutrients]
    reviews = soup.find_all('li', class_='conversation__post')
    print(len(reviews))
    for review in reviews:
        reviewer = review.find('a', class_='post__author-link').text
        review_box = review.find('div', class_='post__text').find('div')
        review = review_box['title']
        if(review == ''):
            review = review_box.find('span', class_='text-truncate__text').text
        reviewer_ids.append(reviewer)
        review_texts.append(review)
        recipe_ids.append(recipe_id)

    review_data = pd.DataFrame({
        'reviewer': reviewer_ids,
        'recipe_id': recipe_ids,
        'review': review_texts
    })

    return (prep_time, serves, ingredients, recipe, nutrients, review_data)

In [121]:
prep_time, serves, ingredients, recipe, nutrients, review_data = scrape_recipe('', '101')

426


In [123]:
review_data

,reviewer,recipe_id,review
0,\n Seashorewalker\n,101,I had chicken breasts to prepare for dinner an...
1,\n t4cram4_7252722\n,101,I finally made this last night. My husband an...
2,\n The Amaze-Ing Me\n,101,Wonderful and easy to make. Used Mrs. dash in...
3,\n jessicaweber1985\n,101,This was sooo good! I used 3 good sized chicke...
4,\n Jonathan Melendez \n,101,Used panko bread crumbs and it was so crunchy ...
...,...,...,...
421,\n soxfan28\n,101,"This was AWESOME!! As others noted, the chicke..."
422,\n Chasealily\n,101,yum yum yuumm!! I am new at cooking and the fa...
423,\n Food Snob \n,101,This chicken was really great~ melt in your mo...
424,\n byacks47\n,101,"Easy to prepare, great alternative to frying a..."
